## Lecture: Python Concurrency
### March 31, 2020  

Part of lecture is based on [https://nyu-cds.github.io/python-concurrency/](https://nyu-cds.github.io/python-concurrency/)
<br>
https://www.geeksforgeeks.org/queue-in-python/

---

**Main goal to explore techniques for improving performance by introducing different forms of concurrency.**


---

For this you will need to:

- create an account in [imgur.com](https://imgur.com/)
- register your application [here](https://api.imgur.com/oauth2/addclient)
  - Authorization Type: __OAuth 2 authorization with a callback URL__
  - Authorization Callback URL: __https://www.getpostman.com/oauth2/callback__
  - email:
  - Description:
  

---
The functions below fetchs a list of images and download them __imgur__ repository: 
[https://imgur.com/](https://imgur.com/)

- We will start with a version that downloads images sequentially, or one at a time

- Then improve the performance by introducing multiprocessing and threading

---
We will split the functionality into three separate functions:
- get_links
- download_link
- setup_download_dir

In [1]:
import json
import os
from pathlib import Path
from urllib.request import urlopen, Request
from time import time

types = {'image/jpeg', 'image/png', 'image/gif'}

def get_links(client_id):
    '''used to obtain a list of available images'''
    headers = {'Authorization': 'Client-ID {}'.format(client_id)}
    req = Request('https://api.imgur.com/3/gallery/random/random/', headers=headers, method='GET')
    
    with urlopen(req) as response:
        data = json.loads(response.read().decode('utf-8'))
    return [item['link'] for item in data['data'] if 'type' in item and item['type'] in types]

def download_link(directory, link):
    '''downloads the image given by the URL _link_ into _directory_'''
    download_path = directory / os.path.basename(link)
    with urlopen(link) as image, download_path.open('wb') as f:
        f.write(image.read())

def setup_download_dir():
    '''creates a download destination directory if it doesn’t already exist'''
    download_dir = Path('images')
    if not download_dir.exists():
        download_dir.mkdir()
    return download_dir

In [2]:
# 'replace with your client ID'
CLIENT_ID = '49142d088194030'

ts = time()
download_dir = setup_download_dir()

links = [l for l in get_links(CLIENT_ID)]

for i, link in enumerate(links):
    print("%2d %s" % (i, link))
    download_link(download_dir, link)

print('Took {}s'.format(time() - ts))



 0 http://i.imgur.com/8VJeSdQh.gif
 1 https://i.imgur.com/8VhvuCN.jpg
 2 https://i.imgur.com/8V4rC.jpg
 3 http://i.imgur.com/8VGPNH3h.gif
 4 https://i.imgur.com/8VoBj0H.gif
 5 http://i.imgur.com/8VbL2sxh.gif
 6 https://i.imgur.com/8VbtBs3.jpg
 7 https://i.imgur.com/8V3ciah.jpg
 8 https://i.imgur.com/8VfN6UE.gif
 9 http://i.imgur.com/8Vjw6JGh.gif
10 https://i.imgur.com/8V90Kod.gif
11 http://i.imgur.com/8VU7HJ5h.gif
12 http://i.imgur.com/8V466Tih.gif
13 https://i.imgur.com/8VhsCzu.jpg
14 https://i.imgur.com/8VwCDvM.jpg
15 https://i.imgur.com/8VengAs.jpg
16 https://i.imgur.com/8VgdD.png
17 https://i.imgur.com/8VVQG.jpg
18 https://i.imgur.com/8VfyT7I.jpg
19 https://i.imgur.com/8Vzvn9a.jpg
20 https://i.imgur.com/8VmYyn8.jpg
21 https://i.imgur.com/8VNXQQ7.png
22 https://i.imgur.com/8VeVEoa.jpg
23 https://i.imgur.com/8VgOB.jpg
24 https://i.imgur.com/8VxgBMK.jpg
25 http://i.imgur.com/8VDgjD3h.gif
26 https://i.imgur.com/8V97Zmu.gif
Took 9.685676097869873s


---

- To improve the performance of the image downloader we can run **multiple copies** of the program at the same time. 


- However, we would need to know what images are available so that we could ensure that one process didn’t download an image that had already been downloaded by a different process.  


- Fortunately the multiprocessing module is available for this purpose.

---

### Pool

- To use multiple processes we need a multiprocessing **Pool**. 


- The Pool class provides a map method that runs a function as a separate process, passing arguments from a supplied iterable. 


- The iterable is divided into a number of chunks, so that each process gets roughly the same number of elements. 


- We will pass the list of URLs to the pool, which starts 8 new processes and use each one to download the images in parallel.

In [1]:
from multiprocessing import cpu_count
print("number of CPU cores:", cpu_count())

number of CPU cores: 4


In [4]:
from functools import partial
from multiprocessing.pool import Pool

def multi_processes_download():
    ts = time()
    download_dir = setup_download_dir()
    links = [l for l in get_links(CLIENT_ID)]

    # functools.partial makes a new version of a function 
    # with one or more fixed arguments
    download = partial(download_link, download_dir)
   
    with Pool(8) as p: #we can also change it to 16, 2 per CPU,
        p.map(download, links)
        
    print('Took {}s'.format(time() - ts))

multi_processes_download()

Took 5.019102573394775s


---

Although easy to implement, the parallelism bears some drawbacks:
- each process contains **a copy of the entire memory**
- it does not handle processes that depend on each other
the local will be local memory for each thred

Those issues can be tackled by shared memory and message passing mechanisms, which we will learn from later lessons.# shared memory will be heap memory

---
## Using Threads

Threading is a well known approach to attaining concurrency: 
- typically threads are lighter weight than processes
- **lower memory requirements**, as **they share the same memory space**

We will relay on __Thread class__, which provides a run method that should be overridden with a method that does the actual work of the thread.

### Thread Safety

- Variables in the program are shared by all the threads and cannot be accessed the way you would normally access a variable. One thread may change the variable while another thread is reading it, or worse, two threads may try to update the variable at the same time. 


- This is known as a **race condition**, it is one of the leading sources of errors in threaded programs and needs to be addressed properly.



- A way to deal with thread safety is using the __Queue Class__

In [2]:
# Understanding Queue 
from queue import Queue

def do_work(q):
    while not q.empty():
        item = q.get()
        print(str(item)) 
        q.task_done()  # this is important when combining Queue with Threads

q = Queue() # FIFO queue

for i in range(20):
    q.put(i)

do_work(q)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


---
A simpler example before going back to the image downloader code

In [3]:
from time import sleep

In [4]:
# in this example each thread prints an element of the queue

from queue import Queue
from threading import Thread
import logging  

logging.basicConfig(format='(%(threadName)-9s) %(message)s',)

def do_work(q):
    while True:
        item = q.get()
        logging.debug("e" + str(item) + ' ')
        print(str(item) + ' ')
        q.task_done()
        sleep(2) 
        #The sleep() function suspends (waits) execution of the current thread for a given number of seconds.
        #However, the function suspends a thread rather than the whole process in multithreaded programs. 
    
q = Queue()
num_threads = 10

for i in range(num_threads):
    worker = Thread(target=do_work, args=(q,), name='thread_' + str(i))
    worker.setDaemon(True) # this stop the threads when the program quits  
    worker.start()         # start the threads

# now we have started 10 threads:

for i in range(50):
    q.put(i)

q.join() # wait untill all threads have finished

0 1 2 3 4 5 6 7 8 9 









10 11 12 13 14 15 16 17 18 19 









20 21 22 23 24 25 26 27 28 29 









30 31 
32 
33 34 
35 36 37 38 39 






40 
41 42 43 


44 
45 46 47 48 

49 




In [8]:
from queue import Queue
from threading import Thread

class DownloadWorker(Thread):
    def __init__(self, queue):
        Thread.__init__(self)
        self.queue = queue
    
    def run(self):
        while True:
            # Get the work from the queue and expand the tuple
            (directory, link) = self.queue.get()
            # call the function donwload_link (previously defined)
            download_link(directory, link)
            self.queue.task_done()

            
def threaded_download():
    ts = time()
    download_dir = setup_download_dir()
    links = [l for l in get_links(CLIENT_ID)]
    
    
    # Create a queue to communicate with the worker threads
    queue = Queue()
    
    # Create 8 worker threads, thread=worker
    for _ in range(8):
        worker = DownloadWorker(queue)
        # Setting daemon to True will let the main thread exit 
        # even though the workers are blocking
        worker.daemon = True
        worker.start()

    
    # Put the tasks into the queue as a tuple
    for link in links:
        print('Queueing: {}'.format(link))
        queue.put((download_dir, link)) 
        # Put an item into the queue. If the queue is full, 
        # wait until a free slot is available before adding the item.
    
    # Causes the main thread to wait for the queue to finish processing all the tasks
    queue.join()
    
    print('Took {}'.format(time() - ts))

threaded_download()

Queueing: http://i.imgur.com/8VJeSdQh.gif
Queueing: https://i.imgur.com/8VhvuCN.jpg
Queueing: https://i.imgur.com/8V4rC.jpg
Queueing: http://i.imgur.com/8VGPNH3h.gif
Queueing: https://i.imgur.com/8VoBj0H.gif
Queueing: http://i.imgur.com/8VbL2sxh.gif
Queueing: https://i.imgur.com/8VbtBs3.jpg
Queueing: https://i.imgur.com/8V3ciah.jpg
Queueing: https://i.imgur.com/8VfN6UE.gif
Queueing: http://i.imgur.com/8Vjw6JGh.gif
Queueing: https://i.imgur.com/8V90Kod.gif
Queueing: http://i.imgur.com/8VU7HJ5h.gif
Queueing: http://i.imgur.com/8V466Tih.gif
Queueing: https://i.imgur.com/8VhsCzu.jpg
Queueing: https://i.imgur.com/8VwCDvM.jpg
Queueing: https://i.imgur.com/8VengAs.jpg
Queueing: https://i.imgur.com/8VgdD.png
Queueing: https://i.imgur.com/8VVQG.jpg
Queueing: https://i.imgur.com/8VfyT7I.jpg
Queueing: https://i.imgur.com/8Vzvn9a.jpg
Queueing: https://i.imgur.com/8VmYyn8.jpg
Queueing: https://i.imgur.com/8VNXQQ7.png
Queueing: https://i.imgur.com/8VeVEoa.jpg
Queueing: https://i.imgur.com/8VgOB.jpg


---

#### Not really parallel !

- Python  has a **Global Interpreter Lock (GIL)**, which makes **one thread to be executed at a time** throughout this process. Therefore, **this code is concurrent but not parallel**. 

- The reason it is still faster is because the image downloader is an input/output (I/O) bound task. 
The majority of the time is spent waiting for the network. This is why threading can provide a large speed increase. 

- **The processor can switch between the threads** whenever one of them is **ready** to do some work.



- If the program was performing a task that was CPU bound, using the threading module in Python or any other interpreted language with a GIL could actually result in reduced performance. 

- For CPU bound tasks and truly parallel execution in Python, the multiprocessing module is a better option.

---

### Sum of array elements in parallel

In [44]:
n = int(1e6)
# n = int(1e4)
# n=30

In [45]:
# Sequencial version
from time import time

ts = time()
s = 0
for i in range(n):
    s = s + i
print(s, '-->', time()-ts,'ms')   

499999500000 --> 0.23294997215270996 ms


In [36]:
chunks_test = [list(range(i,i + 10)) for i in range(0, 30, 10)]
chunks_test

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]]

In [46]:
# multiprocessor version
#this sum can be proceed with paralelle as the sum is determable invariant to the order
# same for multiplication
#but for some calculation, it may not work but some part will work.
# tuhs we will decide which part can be used parallelism

# time a bit longer due to we need to join all queques together
from time import time
from multiprocessing.pool import Pool


def sum_multi_processes(chunk):
    y = 0
    for i in chunk:
        y = y + i
    return(y)

chunks = [list(range(i,i + 100)) for i in range(0, n, 100)]# divudes into cgunks, 0-100, 101-200
# chunks = [list(range(i,i + 10)) for i in range(0, n, 10)]

ts = time()
with Pool(10) as p:
     results = p.map(sum_multi_processes, chunks)

print(sum(results), '-->', time()-ts,'ms')   

499999500000 --> 0.23627090454101562 ms


In [56]:
# Thread version
from queue import Queue
from threading import Thread
from threading import Lock

del total_sum
total_sum = 0
# def sum_chunk(q):
#     while True:
#         global total_sum
#         chunk = q.get()
#         for i in chunk:
#             with Lock():  # force synchronization
#                 total_sum += i
#         q.task_done()

def sum_chunk(q):
    while True:
        global total_sum
        chunk = q.get()
        with Lock():
            for i in chunk:
              # force synchronization
                total_sum += i
        q.task_done()

n = int(1e6)
chunks = [list(range(i, i + 100)) for i in range(0, n, 100)]

ts = time()
q = Queue()
num_threads = 10

for i in range(num_threads):
    worker = Thread(target=sum_chunk, args=(q, ))
    worker.setDaemon(True) # this stop the threads when the program quits  
    worker.start()         # start the threads

for chunk in chunks:
    q.put(chunk)

q.join()
print(total_sum, '-->', time() - ts, 'ms')     

497689521000 --> 0.34532713890075684 ms


### Example: Pi Simultation

In [5]:
import numpy as np

def monte_carlo_pi(n):
    s = 0
    for i in range(n):
        x = np.random.uniform(0, 1)
        y = np.random.uniform(0, 1)
        if (x**2 + y**2) < 1:
            s += 1
    return 4*s/n



In [6]:
%%time
result = [monte_carlo_pi(int(1e6)) for _ in range(10)]
print(result)


[3.1397, 3.141036, 3.140092, 3.141744, 3.141544, 3.141728, 3.138664, 3.14052, 3.142444, 3.141352]
CPU times: user 1min 35s, sys: 278 ms, total: 1min 36s
Wall time: 1min 36s


In [24]:
np.array(result)


array([3.140836, 3.139644, 3.141024, 3.142308, 3.142552, 3.14224 ,
       3.141252, 3.141348, 3.142184, 3.14512 ])

In [25]:
from multiprocessing.pool import Pool


In [26]:
%%time
with Pool(8) as pool:
    result = pool.map(monte_carlo_pi, [int(1e6) for _ in range(10)])

CPU times: user 42.9 ms, sys: 24.8 ms, total: 67.7 ms
Wall time: 5.27 s


In [27]:
np.array(result)

array([3.14078 , 3.14078 , 3.14078 , 3.14078 , 3.14078 , 3.14078 ,
       3.14078 , 3.14078 , 3.141616, 3.141616])